# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [1]:
from rdflib import Namespace, URIRef, RDFS
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK
from buildingmotif.model_builder import TemplateBuilderContext

In [2]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

2025-07-03 11:13:24,515 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/prefix (Name: http://qudt.org/2.1/vocab/prefix). Check this is loaded into BuildingMOTIF
2025-07-03 11:13:24,516 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/bacnet/2020 (Name: http://data.ashrae.org/bacnet/2020). Check this is loaded into BuildingMOTIF
2025-07-03 11:13:24,517 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec/recimports (Name: https://w3id.org/rec/recimports). Check this is loaded into BuildingMOTIF
2025-07-03 11:13:24,518 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/qudt (Name: http://qudt.org/2.1/schema/shacl/qudt). Check this is loaded into BuildingMOTIF
2025-07-03 11:13:24,519 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (Name: http://qudt.org/2.1/vocab/unit). Check this i

In [3]:
brick_tmpl = Library.load(directory='brick-templates')

In [4]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs, reheat_vavs, amt computer spaces]

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)

In [5]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
ahu = ctx['multiple-zone-ahu'](name='ahu1')

# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [6]:
vav = ctx['vav-cooling-only'](name='vav1')
feeds = ctx['feeds'](name = ahu['name'], target = vav['name'])

In [7]:
bldg.add_graph(ctx.compile())

/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "sa_fan-speed, ma_temp, htg_coil-valve_cmd, ra_temp, htg_coil, oa_temp, filter_pd, sa_fan-status, ra_damper, sa_fan, clg_coil, ra_damper-dmppos, sa_temp, sa_fan-start_stop, clg_coil-valve_cmd" were not provided during evaluation
  warnings.warn(
/Users/lazlopaul/Desktop/223p/experiments/graph-pattern-id/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:420: UserWarning: Parameters "dat, zone, dmp-dmppos, occ, co2, dmp, ztemp" were not provided during evaluation
  warnings.warn(


In [8]:
print(bldg.graph.serialize(format='ttl'))

@prefix : <urn:example#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

:ahu1 a brick:AHU ;
    rdfs:label "AHU" ;
    brick:feeds :vav1 ;
    brick:hasPart :clg_coil_627885dc,
        :htg_coil_b855e053,
        :ra_damper_0285ccf7,
        :sa_fan_e6f07b08 ;
    brick:hasPoint :oa_temp_38e41ad2,
        :sa_temp_6d131da9 .

:clg_coil-valve_cmd_7f6985de a brick:Valve_Position_Command ;
    rdfs:label "Valve_Position_Command" .

:clg_coil_627885dc a brick:Cooling_Coil ;
    rdfs:label "Cooling_Coil" ;
    brick:hasPoint :clg_coil-valve_cmd_7f6985de .

:dat_35b92ca0 a brick:Discharge_Air_Temperature_Sensor ;
    rdfs:label "Discharge_Air_Temperature_Sensor" .

:dmp-dmppos_538244b8 a brick:Damper_Position_Command ;
    rdfs:label "Damper_Position_Command" .

:dmp_b7eff8f1 a brick:Damper ;
    rdfs:label "Damper" ;
    brick:hasPoint :dmp-dmppos_538244b8

In [9]:
bldg.graph.serialize('brick-example.ttl', format = 'ttl')

<Graph identifier=a2241a23-cfa5-4f9d-9f7e-8f93a1131505 (<class 'rdflib.graph.Graph'>)>